In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l1, l2

In [3]:
# Set directory paths
dir_black = os.path.join('../Soil types/Black Soil')
dir_Cinder = os.path.join('../Soil types/Cinder Soil')
dir_Laterite = os.path.join('../Soil types/Laterite Soil')
dir_peat = os.path.join('../Soil types/Peat Soil')
dir_yellow = os.path.join('../Soil types/Yellow Soil')

In [4]:
# Image preprocessing and augmentation
image_size = 220
batch_size = 10
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

In [5]:
# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,  # Reduce rotation range
    width_shift_range=0.1,  # Reduce width shift range
    height_shift_range=0.1,  # Reduce height shift range
    shear_range=0.1,  # Reduce shear range
    zoom_range=0.1,  # Reduce zoom range
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [6]:
train_generator = train_datagen.flow_from_directory(
    '../Soil types', 
    target_size=target_size,
    batch_size=batch_size,
    classes=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil'],
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '../Soil types', 
    target_size=target_size,
    batch_size=batch_size,
    classes=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil'],
    class_mode='categorical',
    subset='validation'
)

Found 126 images belonging to 5 classes.
Found 30 images belonging to 5 classes.


In [8]:
# Model architecture
model = tf.keras.models.Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=input_shape),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(512, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dropout(0.4),  # Adjust dropout rate
    
    Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),  # Adjust regularization strength
    Dropout(0.4),  # Adjust dropout rate
    
    Dense(5, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 218, 218, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 109, 109, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 107, 107, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 53, 53, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 51, 51, 256)       295168    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 25, 25, 256)      

In [9]:
# Model compilation
model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=0.0001),  # Adjust learning rate
                metrics=['acc'])

In [10]:
# Callbacks
# Membuat class myCallback untuk menghentikan training ketika akurasi sudah mencapai 86% dan val_acc sudah mencapai 86%
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_acc') > 0.99:
            self.model.stop_training = True

In [11]:
from keras.callbacks import ModelCheckpoint

# Membuat callback untuk menyimpan model
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_epoch_{epoch:03d}.h5',
    monitor='val_acc',  # Memantau akurasi validasi
    save_best_only=True,  # Hanya menyimpan model terbaik
    mode='max',  # Mode menyimpan model (max untuk akurasi)
    verbose=1
)

In [12]:
# Training the model
total_sample = train_generator.n
n_epochs = 200  # Increased number of epochs

In [14]:
# Train the model

callbacks = myCallback()

history = model.fit(
    train_generator,
    steps_per_epoch=int(total_sample / batch_size),
    epochs=n_epochs,
    validation_data=validation_generator,
    validation_steps=int(validation_generator.samples / batch_size),
    callbacks=[callbacks, model_checkpoint_callback],
    verbose=1
)

Epoch 1/200
12/12 [==============================] - ETA: 0s - loss: 1.6862 - acc: 0.2241
Epoch 1: val_acc improved from -inf to 0.20000, saving model to model_epoch_001.h5


/home/khairul/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 13s 1s/step - loss: 1.6862 - acc: 0.2241 - val_loss: 1.6233 - val_acc: 0.2000
Epoch 2/200
12/12 [==============================] - ETA: 0s - loss: 1.6193 - acc: 0.3250
Epoch 2: val_acc improved from 0.20000 to 0.30000, saving model to model_epoch_002.h5
12/12 [==============================] - 12s 987ms/step - loss: 1.6193 - acc: 0.3250 - val_loss: 1.5438 - val_acc: 0.3000
Epoch 3/200
12/12 [==============================] - ETA: 0s - loss: 1.5217 - acc: 0.3362
Epoch 3: val_acc improved from 0.30000 to 0.33333, saving model to model_epoch_003.h5
12/12 [==============================] - 12s 965ms/step - loss: 1.5217 - acc: 0.3362 - val_loss: 1.3087 - val_acc: 0.3333
Epoch 4/200
12/12 [==============================] - ETA: 0s - loss: 1.2772 - acc: 0.3707
Epoch 4: val_acc improved from 0.33333 to 0.40000, saving model to model_epoch_004.h5
12/12 [==============================] - 11s 954ms/step - loss: 1.2772 - acc: 0.3707 - val_loss: 1.0459 - val

In [15]:
from keras.models import load_model

# Misalkan Anda ingin memuat model dari epoch ke-138
epoch_to_load = 138
model_filename = f'model_epoch_{epoch_to_load:03d}.h5'  # Sesuaikan nama berkas model dengan nomor epoch

# Memuat model dari berkas yang sesuai
loaded_model = load_model(model_filename)

# Sekarang Anda memiliki model pada epoch ke-138 dalam variabel loaded_model

In [16]:
# Print class indices
print(train_generator.class_indices)

{'Black Soil': 0, 'Cinder Soil': 1, 'Laterite Soil': 2, 'Peat Soil': 3, 'Yellow Soil': 4}
